In [0]:
!pip install dgl

     |████████████████████████████████| 3.0MB 4.6MB/s 


In [0]:
import dgl
import dgl.function as fn
import torch as th
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph

In [0]:
class GCNLayer(nn.Module):
  def __init__(self,in_feats,out_feats):
    super(GCNLayer,self).__init__()
    self.linear = nn.Linear(in_feats,out_feats)
  
  def forward(self,g,feature):
    with g.local_scope():
      g.ndata['h'] = feature # ndata h에 feature저장
      g.update_all(fn.copy_src(src='h',out='m'),fn.sum(msg='m',out='h'))# src에서 destination으로 sum 한값을 넘김
      h = g.ndata['h']
      return self.linear(h)
  

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layer1 = GCNLayer(1433, 16)
        self.layer2 = GCNLayer(16, 7)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x
print(net)
net = Net()


Net(
  (layer1): GCNLayer(
    (linear): Linear(in_features=1433, out_features=16, bias=True)
  )
  (layer2): GCNLayer(
    (linear): Linear(in_features=16, out_features=7, bias=True)
  )
)


In [0]:
from dgl.data import citation_graph as citegrh
import networkx as nx
def load_cora_data():
    data = citegrh.load_cora()
    features = th.FloatTensor(data.features)
    labels = th.LongTensor(data.labels)
    train_mask = th.BoolTensor(data.train_mask)
    test_mask = th.BoolTensor(data.test_mask)
    g = DGLGraph(data.graph)
    return g, features, labels, train_mask, test_mask

In [0]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with th.no_grad():
        logits = model(g, features)
        logits = logits[mask]
        labels = labels[mask]
        _, indices = th.max(logits, dim=1)#가장 큰값들을 뽑아냄
        correct = th.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [0]:
import time
import numpy as np
g, features, labels, train_mask, test_mask = load_cora_data()
optimizer = th.optim.Adam(net.parameters(), lr=1e-2)
dur = []

for epoch in range(50):
    if epoch >=3:
        t0 = time.time()

    net.train()
    logits = net(g, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp[train_mask], labels[train_mask])
    #loss구하기

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #update
    
    if epoch >=3:
        dur.append(time.time() - t0)

    acc = evaluate(net, g, features, labels, test_mask)
    print("Epoch {:05d} | Loss {:.4f} | Test Acc {:.4f} | Time(s) {:.4f}".format(
            epoch, loss.item(), acc, np.mean(dur)))

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Loss 0.0242 | Test Acc 0.7850 | Time(s) nan
Epoch 00001 | Loss 0.0220 | Test Acc 0.7520 | Time(s) nan
Epoch 00002 | Loss 0.1013 | Test Acc 0.7790 | Time(s) nan
Epoch 00003 | Loss 0.0197 | Test Acc 0.7940 | Time(s) 0.1420
Epoch 00004 | Loss 0.0376 | Test Acc 0.7900 | Time(s) 0.1424
Epoch 00005 | Loss 0.0592 | Test Acc 0.7860 | Time(s) 0.1432
Epoch 00006 | Loss 0.0683 | Test Acc 0.7790 | Time(s) 0.1440
Epoch 00007 | Loss 0.0658 | Test Acc 0.7780 | Time(s) 0.1439
Epoch 00008 | Loss 0.0586 | Test Acc 0.7660 | Time(s) 0.1445
Epoch 00009 | Loss 0.0496 | Test Acc 0.7580 | Time(s) 0.1447
Epoch 00010 | Loss 0.0384 | Test Acc 0.7570 | Time(s) 0.1448
Epoch 00011 | Loss 0.0256 | Test Acc 0.7560 | Time(s) 0.1446
Epoch 00012 | Loss 0.0161 | Test Acc 0.7460 | Time(s) 0.1447
Epoch 00013 | Loss 0.0218 | Test Acc 0.7430 | Time(s) 0.1450
Epoch 00014 | Loss 0.0361 | Test Acc 0.7460 | Time(s) 0.1450
Epoch 00015 | Loss 0.0346 | Test Acc 0.7530 | Time(s) 0.1457
Epoch 00016 | Loss 0.0213 | Test 